In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf)

# RDD 생성
일반 파이썬의 리스트를 이용해서 RDD 생성
* `parallelize([item1, item2, item3, ...])`

In [2]:
foods = sc.parallelize([
    "짜장면", "짬뽕", "마라탕",
    "떡볶이", "쌀국수","짬뽕",
    "짜장면", "짜장면", "짬뽕",
    "마라탕", "라면", "라면",
    "우동", "쌀국수"
])
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수']

각 음식 별 개수 세기
* `countByValue()`

In [3]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 3,
             '짬뽕': 3,
             '마라탕': 2,
             '떡볶이': 1,
             '쌀국수': 2,
             '라면': 2,
             '우동': 1})

상위 `n`개의 데이터 가져오기
* `take(n)`
* `pandas`의 `DataFrame`의 `head`와 흡사

In [4]:
foods.take(3)

['짜장면', '짬뽕', '마라탕']

In [5]:
foods.take(10)

['짜장면', '짬뽕', '마라탕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짬뽕', '마라탕']

처음 1개의 데이터 가져오기
* `first()`

In [6]:
foods.first()

'짜장면'

RDD 내 전체 데이터의 개수 세기
* `count()`

In [7]:
foods.count()

14

중복 데이터 제거
* `distinct()`
* `Transformation`

In [8]:
fd = foods.distinct()
fd

PythonRDD[10] at RDD at PythonRDD.scala:53

In [9]:
fd.collect()

['짜장면', '짬뽕', '마라탕', '떡볶이', '쌀국수', '라면', '우동']

In [10]:
# 중복을 제외한 데이터 개수
fd.count()

7

# Narrow Transformations
* 1:1 변환을 의미한다.
* 하나의 열을 조작하기 위해서 다른 열 및 파티션의 데이터를 사용하지 않는다.

`map(<task>)`
- 데이터를 하나씩 꺼내서 `<task>`가 적용된 새로운 RDD가 만들어 진다.

In [11]:
sample_rdd = sc.parallelize([1, 2, 3])
sample_rdd

ParallelCollectionRDD[12] at readRDDFromFile at PythonRDD.scala:274

In [12]:
# task 함수는 반드시 리턴이 있어야 한다.
sample_rdd2 = sample_rdd.map(lambda x : x + 2)
sample_rdd2

PythonRDD[13] at RDD at PythonRDD.scala:53

In [13]:
sample_rdd2.collect()

[3, 4, 5]

`flatMap(<task>)`
- `map` 함수와 거의 비슷하나, `flatMap`함수는 `map`의 모든 결과를 1차원 배열 형식으로 평평하게(`flat`)하게 나타낸다.

In [14]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [15]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[14] at readRDDFromFile at PythonRDD.scala:274

In [16]:
mapMovies = moviesRDD.map(lambda x : x.split())
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [17]:
flatMovies = moviesRDD.flatMap(lambda x : x.split())
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

조건에 맞는 결과만 얻어내기
* `filter(<task>)`
* `task`의 결과가 True인 데이터만 추출된다.

In [18]:
filteredMovie = flatMovies.filter(lambda x : x != '매트릭스')
filteredMovie.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

## 집합 `Transformations`

In [19]:
num1 = sc.parallelize([1, 2, 3, 4, 5])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

교집합 - `intersection`

In [20]:
intersecRDD = num1.intersection(num2)
intersecRDD

PythonRDD[27] at RDD at PythonRDD.scala:53

In [21]:
intersecRDD.collect()

[4, 5]

합집합 - `union`

In [22]:
unionRDD = num1.union(num2)
unionRDD.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

차집합 - `subtract`

In [23]:
subRDD = num1.subtract(num2) # num1 - num2
subRDD.collect()

[2, 1, 3]

In [24]:
num2.subtract(num1).collect() # num2 - num1

[6, 8, 10, 7, 9]

## Wide Transformations
* `groupBy(<task>)`
    * `task` : 데이터를 묶어주는(Grouping) 기준을 설정

In [30]:
foods = sc.parallelize([
    "짜장면", "짬뽕", "마라탕",
    "떡볶이", "쌀국수","짬뽕",
    "짜장면", "짜장면", "짬뽕",
    "마라탕", "라면", "라면",
    "우동", "쌀국수", "짬뽕밥", "짬짜면", "볶음밥", "볶짬면"
])
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수',
 '짬뽕밥',
 '짬짜면',
 '볶음밥',
 '볶짬면']

In [31]:
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수',
 '짬뽕밥',
 '짬짜면',
 '볶음밥',
 '볶짬면']

In [32]:
# 그룹핑의 기준을 문자열의 첫 번째 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

PythonRDD[57] at RDD at PythonRDD.scala:53

In [33]:
result = foodsGroup.collect()
result

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f0dae319fd0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f0dae319710>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f0dae319990>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f0dae319a90>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f0dae319a50>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f0dae319110>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f0dae319b10>),
 ('볶', <pyspark.resultiterable.ResultIterable at 0x7f0dae319e50>)]

In [34]:
for k, v in result:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕', '짬뽕', '짬뽕밥', '짬짜면']
마 ['마라탕', '마라탕']
떡 ['떡볶이']
쌀 ['쌀국수', '쌀국수']
라 ['라면', '라면']
우 ['우동']
볶 ['볶음밥', '볶짬면']


In [35]:
sc.stop()